# Task 6.1 Set Estimator Options

## Objective 1: Estimator Options

The `Options` class for EstimatorV2 provides various configuration settings to `Estimator` execution .It impacts performance, accuracy, and resource usage.

### Attributes


| Attribute | Description | Typical Values |
|-----------|-------------|----------------|
| **`_VERSION`** | Internal version tracking | `2` (for EstimatorV2) |
| **`max_execution_time`** | Maximum time (seconds) for job execution | `300` (5 min), `3600` (1 hour) |
| **`environment`** | Execution environment metadata | `{'module': 'qiskit_ibm_runtime'}` |
| **`simulator`** | Simulator-specific options | `{'noise_model': None, 'seed_simulator': None}` |
| **`default_precision`** | Target precision for expectation values | `0.01`, `0.001` (smaller = more accurate) |
| **`default_shots`** | Default number of circuit executions | `1024`, `4096`, `8192` |
| **`resilience_level`** | Level of error mitigation | `0` (none) to `3` (maximum) |
| **`seed_estimator`** | Random seed for reproducible results | Integer value (e.g., `42`) |
| **`dynamical_decoupling`** | Enable/configure dynamical decoupling | Boolean or configuration dict |
| **`resilience`** | Advanced resilience configuration | `Options.ResilienceOptionsV2` object |
| **`execution`** | Execution-specific settings | Various execution parameters |
| **`twirling`** | Gate twirling configuration | `Options.TwirlingOptions` object |
| **`experimental`** | Experimental features | Dictionary of experimental settings |

In [ ]:
# Example: Configuring Estimator Options

from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator, EstimatorOptions as Options
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Initialize service (assuming credentials is already saved)
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Create a simple test circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
observable = SparsePauliOp("ZZ")

# Create Options object with custom settings
options = Options()

# Set basic execution parameters
options.max_execution_time = 300
options.default_shots = 2048 
options.default_precision = 0.01
options.seed_estimator = 12345

# Set resilience level (0=none, 1=basic, 2=moderate, 3=maximum)
options.resilience_level = 2  # moderate error mitigation

print(f"Max execution time: {options.max_execution_time} seconds")
print(f"Default shots: {options.default_shots}")
print(f"Default precision: {options.default_precision}")
print(f"Resilience level: {options.resilience_level}\n")

# Create Estimator with custom options
estimator = Estimator(mode=backend, options=options)

print("Updating Options\n")

# Create initial options
initial_options = Options()
initial_options.default_shots = 1024
initial_options.max_execution_time = 300

print(f"Initial shots: {initial_options.default_shots}")
print(f"Initial max time: {initial_options.max_execution_time}")

# Update options with new values
initial_options.update(default_shots=4096, max_execution_time=900)

print(f"Updated shots: {initial_options.default_shots}")
print(f"Updated max time: {initial_options.max_execution_time}\n")

### Methods


#### update

Updates multiple options at once. This is useful for modifying options after creating an Estimator instance or for applying configuration changes.

In [ ]:
options = Options()
options.update(
    default_shots=4096,
    max_execution_time=1200,
    resilience_level=3
)

## Objective 2: Twirling Options

Twirling (or randomized compiling) is a technique that averages over different but equivalent gate sequences to transform coherent errors into stochastic errors, which are easier to mitigate.

**Types:**

1. **Gate Twirling**: Applying random but logically equivalent gate sequences
2. **Pauli Twirling**: Applying random Pauli gates before and after operations
3. **Benefits**: Converts coherent errors into stochastic noise, improves error mitigation effectiveness

## Objective 3: Resilience Options

Resilience options control error mitigation techniques in EstimatorV2. These settings help improve the accuracy of quantum computations on noisy hardware.


### Attributes


| Attribute | Description | Impact |
|-----------|-------------|---------|
| **`measure_mitigation`** | Corrects measurement errors | Improves readout fidelity |
| **`measure_noise_learning`** | Learns measurement noise model | Enables adaptive mitigation |
| **`zne_mitigation`** | Enables Zero-Noise Extrapolation | Reduces gate error effects |
| **`zne`** | ZNE-specific configuration | Fine-tunes extrapolation |
| **`pec_mitigation`** | Enables Probabilistic Error Cancellation | Corrects gate errors |
| **`pec`** | PEC-specific configuration | Controls cancellation parameters |
| **`layer_noise_learning`** | Learns layer-wise noise | Improves noise modeling |
| **`layer_noise_model`** | Custom layer noise model | User-provided noise data |


In [ ]:
from qiskit_ibm_runtime.options import MeasureNoiseLearningOptions,LayerNoiseLearningOptions,ZneOptions

# Example: Configuring Resilience Options


# Method 1: Using resilience_level (simplified)
levels = [0, 1, 2]
descriptions = [
    "No error mitigation",
    "Minimal measurement error mitigation",
    "Medium mitigation"
]

for level, desc in zip(levels, descriptions):
    options = Options()
    options.resilience_level = level
    print(f"Level {level}: {desc}")
    print(f"  Shots may increase: {level * 2}x (approx.)")
    print(f"  Execution time: {level * 1.5}x (approx.)\n")

# Method 2: Fine-grained control with ResilienceOptionsV2

options = Options()

# Configure individual resilience techniques
# Basic readout correction
options.resilience.measure_mitigation = True
# Learn measurement noise
options.resilience.measure_noise_learning = MeasureNoiseLearningOptions()
# Enable Zero-Noise Extrapolation
options.resilience.zne_mitigation = True
# Disable PEC (computationally expensive)
options.resilience.pec_mitigation = False
# Learn layer-dependent noise
options.resilience.layer_noise_learning = LayerNoiseLearningOptions() 

# Configure ZNE specifically
options.resilience.zne = ZneOptions(noise_factors=[1.0, 2.0, 3.0], extrapolator="exponential")

print("Fine-grained resilience settings:")
print(f"  Measurement mitigation: {options.resilience.measure_mitigation}")
print(f"  Measurement noise learning: {options.resilience.measure_noise_learning}")
print(f"  ZNE mitigation: {options.resilience.zne_mitigation}")
print(f"  PEC mitigation: {options.resilience.pec_mitigation}")
print(f"  Layer noise learning: {options.resilience.layer_noise_learning}")
print(f"  ZNE noise factors: {options.resilience.zne.noise_factors}")
print(f"  ZNE extrapolator: {options.resilience.zne.extrapolator}\n")


# Phase 1: minimal resilience
quick_test = Options()
quick_test.resilience_level = 0
quick_test.default_shots = 256
print(f"  Resilience: {quick_test.resilience_level} (none)")
print(f"  Shots: {quick_test.default_shots}")

# Phase 2: moderate accuracy
moderate = Options()
moderate.resilience_level = 1
moderate.default_shots = 1024
print(f"  Resilience: {moderate.resilience_level} (basic)")
print(f"  Shots: {moderate.default_shots}")

# Phase 3: High accuracy (production)
production = Options()
production.resilience_level = 2
production.default_shots = 4096
print(f"  Resilience: {production.resilience_level} (advanced)")
print(f"  Shots: {production.default_shots}")

## Objective 4: ZNE Options

### Attributes

* ampllifier --> gate_folding , gate_folding_front, gate_folding_back, pea
* noise_factors
* extrapolator
* extrapolated_noise_factors